<h1> Imports </h1>

In [1]:
import requests
import uuid
from datetime import datetime
from typing import Dict, Any, Optional, List

<h1> Estrutura Base para usar os endpoints de contexto </h1>

In [2]:
# Configuração base
BASE_URL = "http://212.85.1.27:8009/"  # Ajuste para sua URL

def extrair_telefones_periodo(
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None
) -> Dict[str, Any]:
    """
    Extrai telefones que tiveram conversas no período
    
    Args:
        data_inicio: 'YYYY-MM-DD HH:MM:SS' ou None
        data_fim: 'YYYY-MM-DD HH:MM:SS' ou None
    """
    url = f"{BASE_URL}/context/extrair-telefones-conversas-periodo"
    payload = {}
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def extrair_telefones_com_ia(
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None
) -> Dict[str, Any]:
    """
    Extrai telefones que tiveram mensagens de IA no período
    """
    url = f"{BASE_URL}/context/extrair-telefones-conversas-periodo-com-ia"
    payload = {}
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def buscar_conversa_numero(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    quebrar_conversas: bool = False,
    formato_saida: str = "texto"
) -> Dict[str, Any]:
    """
    Busca conversa de um número específico com fallback
    
    Args:
        numero_telefone: Número do WhatsApp
        formato_saida: "texto" ou "json"
    """
    url = f"{BASE_URL}/context/buscar-conversas-numero-com-fallback"
    payload = {
        "numero_telefone": numero_telefone,
        "quebrar_conversas": quebrar_conversas,
        "formato_saida": formato_saida,
        "forcar_evolution_apenas": True
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def analisar_quantitativo(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    teto_minutos: int = 30
) -> Dict[str, Any]:
    """
    Análise quantitativa de uma conversa
    """
    url = f"{BASE_URL}/context/analisar-conversa-atributos-quantitativos"
    payload = {
        "numero_telefone": numero_telefone,
        "teto_minutos": teto_minutos
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    
    response = requests.post(url, json=payload)
    return response.json()

def analisar_qualitativo(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    campos_analise: Optional[List[str]] = None,
    temperatura: float = 0.1,
    modelo: Optional[str] = None
) -> Dict[str, Any]:
    """
    Análise qualitativa de uma conversa com LLM
    
    Args:
        campos_analise: Lista de campos para analisar ou None para padrão
    """
    url = f"{BASE_URL}/context/analisar-conversa-atributos-qualitativos"
    payload = {
        "numero_telefone": numero_telefone,
        "temperatura": temperatura
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    if campos_analise:
        payload["campos_analise"] = campos_analise
    if modelo:
        payload["modelo"] = modelo
    
    response = requests.post(url, json=payload)
    return response.json()

def obter_memoria_contexto(
    numero_telefone: str,
    data_inicio: Optional[str] = None,
    data_fim: Optional[str] = None,
    teto_minutos: int = 30,
    temperatura: float = 0.1,
    modelo: Optional[str] = None
) -> Dict[str, Any]:
    """
    Análise completa (quantitativa + qualitativa fusionada)
    """
    url = f"{BASE_URL}/context/obter-memoria-contexto"
    payload = {
        "numero_telefone": numero_telefone,
        "teto_minutos": teto_minutos,
        "temperatura": temperatura
    }
    
    if data_inicio:
        payload["data_hora_inicio"] = data_inicio
    if data_fim:
        payload["data_hora_fim"] = data_fim
    if modelo:
        payload["modelo"] = modelo
    
    response = requests.post(url, json=payload)
    return response.json()

In [3]:
def schedule_birthday_task(
    target_datetime: str,
    phone_number: str, 
    nome_pessoa: str,
    tipo_fluxo: str = "normal",
    base_url: str = "http://212.85.1.27:8009",
    custom_task_id: Optional[str] = None
) -> Dict[str, Any]:
    """
    Agenda uma task de aniversário para execução automática
    
    Args:
        target_datetime (str): Data/hora de execução no formato "2025-12-25T09:00:00"
        phone_number (str): Número de telefone no formato "5531999999999"
        nome_pessoa (str): Nome da pessoa que fará aniversário
        tipo_fluxo (str): Tipo do fluxo ("normal" ou "final_de_semana")
        base_url (str): URL base da API (opcional)
        custom_task_id (str): ID personalizado da task (opcional)
    
    Returns:
        Dict com resultado do agendamento
        
    Examples:
        >>> # Agendar aniversário para amanhã às 9h
        >>> result = schedule_birthday_task(
        ...     target_datetime="2025-08-19T09:00:00",
        ...     phone_number="5531995655690", 
        ...     nome_pessoa="João Silva",
        ...     tipo_fluxo="normal"
        ... )
        >>> print(result["status"])
        "success"
        
        >>> # Agendar para final de semana
        >>> result = schedule_birthday_task(
        ...     target_datetime="2025-12-21T10:00:00",
        ...     phone_number="5531888888888",
        ...     nome_pessoa="Maria Santos", 
        ...     tipo_fluxo="final_de_semana"
        ... )
    """
    
    # Gerar ID único se não fornecido
    if not custom_task_id:
        task_id = f"aniversario_{nome_pessoa.replace(' ', '_').lower()}_{uuid.uuid4().hex[:8]}"
    else:
        task_id = custom_task_id
    
    # Montar payload da requisição
    payload = {
        "target_datetime": target_datetime,
        "task_type": "http_request", 
        "task_id": task_id,
        "description": f"Aniversário de {nome_pessoa} ({tipo_fluxo})",
        "task_params": {
            "url": f"{base_url}/activation/trigger",
            "method": "POST",
            "payload": {
                "phone_number": phone_number,
                "flow": "aniversarios",
                "informacoes_contexto": {
                    "nome_pessoa": nome_pessoa,
                    "tipo_fluxo": tipo_fluxo
                }
            }
        }
    }
    
    # URL do endpoint de agendamento
    schedule_url = f"{base_url}/tasks/schedule"
    
    print(f"🎂 [SCHEDULE-BIRTHDAY] Agendando aniversário de {nome_pessoa}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Para: {target_datetime}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Phone: {phone_number}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Tipo: {tipo_fluxo}")
    print(f"🎂 [SCHEDULE-BIRTHDAY] Task ID: {task_id}")
    
    try:
        # Fazer requisição
        response = requests.post(
            schedule_url,
            json=payload,
            headers={'Content-Type': 'application/json'},
            timeout=30
        )
        
        # Verificar se foi bem-sucedida
        if response.status_code == 200:
            result = response.json()
            
            print(f"✅ [SCHEDULE-BIRTHDAY] Sucesso! Task {task_id} agendada")
            print(f"✅ [SCHEDULE-BIRTHDAY] Próxima execução: {result.get('next_run', 'N/A')}")
            
            return {
                "status": "success",
                "task_id": task_id,
                "nome_pessoa": nome_pessoa,
                "phone_number": phone_number,
                "scheduled_for": target_datetime,
                "tipo_fluxo": tipo_fluxo,
                "api_response": result,
                "message": f"Aniversário de {nome_pessoa} agendado com sucesso!"
            }
        else:
            error_data = response.json() if response.content else {}
            
            print(f"❌ [SCHEDULE-BIRTHDAY] Erro HTTP {response.status_code}")
            print(f"❌ [SCHEDULE-BIRTHDAY] Resposta: {error_data}")
            
            return {
                "status": "error",
                "error": f"HTTP {response.status_code}",
                "task_id": task_id,
                "nome_pessoa": nome_pessoa,
                "phone_number": phone_number,
                "api_response": error_data,
                "message": f"Erro ao agendar aniversário de {nome_pessoa}"
            }
            
    except requests.exceptions.RequestException as e:
        print(f"❌ [SCHEDULE-BIRTHDAY] Erro de conexão: {e}")
        
        return {
            "status": "error",
            "error": str(e),
            "task_id": task_id,
            "nome_pessoa": nome_pessoa,
            "phone_number": phone_number,
            "message": f"Erro de conexão ao agendar aniversário de {nome_pessoa}"
        }
    
    except Exception as e:
        print(f"❌ [SCHEDULE-BIRTHDAY] Erro inesperado: {e}")
        
        return {
            "status": "error",
            "error": str(e),
            "task_id": task_id,
            "nome_pessoa": nome_pessoa,
            "phone_number": phone_number,
            "message": f"Erro inesperado ao agendar aniversário de {nome_pessoa}"
        }

<h1> Grupos </h1>

<h1> Análise </h1>

In [18]:
import requests

import requests

from urllib.parse import quote

import re

def fetch_all_groups(base_url: str, instance: str, api_key: str, get_participants: bool = False):
    """
    Busca todos os grupos de uma instância.
    """
    url = f"{base_url}/group/fetchAllGroups/{instance}"
    params = {"getParticipants": str(get_participants).lower()}
    headers = {
        "apikey": api_key,            # 👈 exatamente como no Postman
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)

    # Se a API retornar 401, mostra detalhes
    if response.status_code == 401:
        raise Exception(f"Unauthorized (401). Verifique se o API Key está correto. "
                        f"URL: {response.url}, Headers enviados: {headers}")

    response.raise_for_status()
    return response.json()

def find_group_participants(base_url: str, instance: str, api_key: str, group_jid: str):
    """
    Busca os participantes de um grupo específico.

    :param base_url: URL base da API (ex: https://api.exemplo.com)
    :param instance: Identificador da instância
    :param api_key: Chave da API
    :param group_jid: JID do grupo (ex: "554899999999-123456@g.us")
    :return: Resposta JSON da API
    """
    # Codificar instance para garantir compatibilidade (ex: espaços)
    instance_encoded = quote(instance, safe='')
    
    url = f"{base_url}/group/participants/{instance_encoded}"
    params = {"groupJid": group_jid}
    headers = {
        "apikey": api_key,
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 401:
        raise Exception(f"Unauthorized (401). Verifique se o API Key está correto. "
                        f"URL: {response.url}, Headers enviados: {headers}")

    response.raise_for_status()
    return response.json()

def find_messages(base_url, instance, api_key, remote_jid):
   """
   Busca mensagens na Evolution API
   
   Args:
       base_url (str): URL base da API (ex: 'https://api.exemplo.com')
       instance (str): Nome da instância
       api_key (str): Chave da API
       remote_jid (str): ID remoto do contato/grupo
   
   Returns:
       dict: Resposta da API
   """
   
   url = f"{base_url}/chat/findMessages/{instance}"
   
   headers = {
       "apikey": api_key,
       "Content-Type": "application/json"
   }
   
   body = {
       "where": {
           "key": {
               "remoteJid": remote_jid
           }
       }
   }
   
   try:
       response = requests.post(url, headers=headers, json=body)
       response.raise_for_status()
       return response.json()
   
   except requests.exceptions.RequestException as e:
       print(f"Erro na requisição: {e}")
       return None

# Exemplo de uso:
# result = find_messages("https://sua-api.com", "minha_instancia", "sua_api_key", "5511999999999@s.whatsapp.net")

def format_brazilian_phone_to_whatsapp(phone_number):
    clean_number = ''.join(filter(str.isdigit, phone_number))
    
    # Se tem 13 dígitos, remove o 5º dígito (9 extra)
    if len(clean_number) == 13 and clean_number.startswith('55'):
        formatted_number = clean_number[:4] + clean_number[5:]
    else:
        formatted_number = clean_number
    
    return f"{formatted_number}@s.whatsapp.net"

def limpar_numero_whatsapp(numero: str) -> str:
    """Converte '553195655690@s.whatsapp.net' para '5531995655690'."""
    numero_limpo = numero.split("@")[0]  # remove @s.whatsapp.net
    if len(numero_limpo) > 4 and numero_limpo[4] != "9":
        numero_limpo = numero_limpo[:4] + "9" + numero_limpo[4:]
    return numero_limpo

def find_correct_brazilian_phone_format(base_url, instance, api_key, phone_number):
    """
    Descobre o formato correto do número brasileiro para a Evolution API
    
    Returns:
        str: remoteJid no formato correto ou None se não encontrar mensagens
    """
    
    def find_messages(remote_jid):
        import requests
        from urllib.parse import quote
        
        encoded_instance = quote(instance)
        url = f"{base_url}/chat/findMessages/{encoded_instance}"
        
        headers = {
            "apikey": api_key,
            "Content-Type": "application/json"
        }
        
        body = {
            "where": {
                "key": {
                    "remoteJid": remote_jid
                }
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição para {remote_jid}: {e}")
            return None
    
    clean_number = ''.join(filter(str.isdigit, phone_number))
    
    # Define os formatos a testar
    formats = []
    if len(clean_number) == 13 and clean_number.startswith('55'):
        formats = [
            f"{clean_number}@s.whatsapp.net",  # 13 dígitos primeiro
            f"{clean_number[:4]}{clean_number[5:]}@s.whatsapp.net"  # 12 dígitos
        ]
    else:
        formats = [f"{clean_number}@s.whatsapp.net"]
    
    # Testa cada formato
    for remote_jid in formats:
        result = find_messages(remote_jid)
        if result and result.get('messages', {}).get('records') and len(result['messages']['records']) > 0:
            return remote_jid  # Retorna apenas o formato correto
    
    return None  # Não encontrou mensagens em nenhum formato

def check_phone_has_messages_raw(base_url, instance, api_key, phone_number):
    """
    Verifica se um número tem mensagens SEM fazer conversão de formato
    Testa o número exatamente como foi fornecido
    
    Args:
        base_url (str): URL base da API
        instance (str): Nome da instância
        api_key (str): Chave da API
        phone_number (str): Número de telefone (sem conversão)
    
    Returns:
        bool: True se encontrar mensagens, False caso contrário
    """
    
    def find_messages(remote_jid):
        """
        Inner function para buscar mensagens na Evolution API
        """
        import requests
        from urllib.parse import quote
        
        encoded_instance = quote(instance)
        url = f"{base_url}/chat/findMessages/{encoded_instance}"
        
        headers = {
            "apikey": api_key,
            "Content-Type": "application/json"
        }
        
        body = {
            "where": {
                "key": {
                    "remoteJid": remote_jid
                }
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException:
            return None
    
    # Usa o número exatamente como foi fornecido + @s.whatsapp.net
    clean_number = ''.join(filter(str.isdigit, phone_number))
    remote_jid = f"{clean_number}@s.whatsapp.net"
    
    # Testa apenas o formato fornecido, sem conversões
    result = find_messages(remote_jid)
    if result and result.get('messages', {}).get('records') and len(result['messages']['records']) > 0:
        return True
    
    return False


import requests
import json

def send_message(numero: str, mensagem: str) -> dict:
    url = "http://212.85.1.27:8025/chat/send"
    headers = {"Content-Type": "application/json"}
    payload = {
        "phone_number": numero,
        "message": mensagem,
        "type": "text"
    }

    # aqui forçamos a serialização como no curl -d '...'
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    
    try:
        return response.json()
    except Exception:
        return {"status": response.status_code, "response": response.text}

# Agora testando seu exemplo:
# check_phone_has_messages_raw(..., "5531995655690")  # → Deveria ser False
# check_phone_has_messages_raw(..., "553195655690")   # → Deveria ser True

In [7]:
resultados = fetch_all_groups("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm")

In [8]:
resultados

[{'id': '120363416579819696@g.us',
  'subject': 'Comunidade Tô Rica',
  'subjectOwner': '553798694711@s.whatsapp.net',
  'subjectTime': 1743685694,
  'pictureUrl': None,
  'size': 4,
  'creation': 1743685694,
  'owner': '553798694711@s.whatsapp.net',
  'desc': 'Comunidade Tô Rica',
  'descId': '10100.64064-332',
  'restrict': False,
  'announce': False,
  'isCommunity': True,
  'isCommunityAnnounce': False},
 {'id': '120363338050597260@g.us',
  'subject': 'Mindfullnes ✨️ Num Instante',
  'subjectOwner': '553198861210@s.whatsapp.net',
  'subjectTime': 1730720420,
  'pictureUrl': None,
  'size': 1,
  'creation': 1730420716,
  'owner': '553198861210@s.whatsapp.net',
  'desc': 'Mindfulness dia a dia, com Dra. Thaís Santos.',
  'descId': '846418167FEA4BCAF00339F4FA847661',
  'restrict': False,
  'announce': False,
  'isCommunity': True,
  'isCommunityAnnounce': False},
 {'id': '120363267261091493@g.us',
  'subject': 'RBC NETWORKING',
  'subjectOwner': '553192670015@s.whatsapp.net',
  'subje

In [18]:

for resultado in resultados:
    print(resultado['subject'], resultado['id'])


Comunidade Tô Rica 120363416579819696@g.us
Mindfullnes ✨️ Num Instante 120363338050597260@g.us
RBC NETWORKING 120363267261091493@g.us
Clube de Descontos PURE Blends (Comunidade Exclusiva Para Clientes Pure Blends) 2 120363183033245439@g.us
GCRIE 120363138329335509@g.us
COMUNIDADE N 120363029089686299@g.us
Team Comunidade 553199586356-1625754390@g.us
PROVA SOCIAL SAFIRA 553199586356-1620831995@g.us
LINK WEBNARIO HOJE 15HRS 553199586356-1606138252@g.us
LANÇAMENTO MARCA FM 120363040979925061@g.us
FOTOS LUAN 553199586356-1622290293@g.us
😍PASTA ESPECIAL😍 TRABALHO 553199586356-1604746828@g.us
VENDAS 120363040020551019@g.us
VIDEO ENCONTRO 120363020382343961@g.us
SAFIRA ACELERAÇÃO 2023 120363044762317191@g.us
ALAVANCADA SAFIRA - FALTAM 56 DIAS 120363044266045451@g.us
#EUSOUSAFIRA 💎 5521973093879-1620396297@g.us
PLANEJA 553194929825-1625607459@g.us
CONSULTORIA ROYAL PRESTIGE  120363301807227576@g.us
Palestrantes da Noite 120363326293571862@g.us
[ GT MARKETING ] - Fábricia 120363319589600946@g.u

In [16]:
participantes_grupo = find_group_participants("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm",
                 "120363344594416817@g.us")

In [17]:
participantes_grupo

{'participants': [{'id': '553199101795@s.whatsapp.net',
   'jid': '553199101795@s.whatsapp.net',
   'lid': '42537490354178@lid',
   'admin': None,
   'name': 'Luciana Prates',
   'imgUrl': 'https://pps.whatsapp.net/v/t61.24694-24/505186310_1670592770282141_2133193815112404566_n.jpg?ccb=11-4&oh=01_Q5Aa2QGY6miZiobfJ5-Uo1OSQEM9KyduagVb5xayQJjErORlfQ&oe=68BC5206&_nc_sid=5e03e0&_nc_cat=103'},
  {'id': '553186774435@s.whatsapp.net',
   'jid': '553186774435@s.whatsapp.net',
   'lid': '263011012985082@lid',
   'admin': None,
   'name': 'Roselayne',
   'imgUrl': 'https://pps.whatsapp.net/v/t61.24694-24/491841935_1372334223794067_4794234634606846587_n.jpg?ccb=11-4&oh=01_Q5Aa2QGMWti2Xhrn1mhlwUnMjtX1UYZFn2wts6GExwi49LY8DA&oe=68BCAB76&_nc_sid=5e03e0&_nc_cat=100'},
  {'id': '553185051528@s.whatsapp.net',
   'jid': '553185051528@s.whatsapp.net',
   'lid': '195202035806413@lid',
   'admin': None},
  {'id': '553188858481@s.whatsapp.net',
   'jid': '553188858481@s.whatsapp.net',
   'lid': '2135115316757

<h1> Números que a Letícia Enviou/Ativou </h1>

In [15]:
telefones_ja_ativados = [
    "5527992027535",
    "5524981604432",
    "553196856817",
    "553196161702",
    "553197733587",
    "553190861806",
    "555599666832",
    "553172144220",
    "553183993170",
    "553188683543",
    "5527997374177",
    "553171487572",
    "553171700813",
    "553173438496",
    "553173451644",
    "553183160550",
    "553185588939",
    "553188929472",
    "553189027712",
    "553190670771",
    "553191004550",
    "553192125169",
    "553194667082",
    "553195693255",
    "553197013863",
    "553197960044",
    "553198795412",
    "556181353474",
    "553199823391",
    "553199924241",
    "553288071604",
    "553598125948",
    "555195000146",
    "555195056289",
]


In [41]:
len(telefones_ja_ativados)

34

In [5]:
find_correct_brazilian_phone_format("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm", "5531995655690")

'553195655690@s.whatsapp.net'

<h1> Total Base de Contatos do Grupo MasterClass </h1>

In [19]:
telefones_convertidos_jid = [find_correct_brazilian_phone_format("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm", tel) for tel in telefones_ja_ativados]

dados_participantes = {dados.get('id'): {'name': dados.get('name', None),
                                         }
                       for dados in participantes_grupo['participants']}

dados_participantes_filtrado = {
    jid: info
    for jid, info in dados_participantes.items()
    if jid not in telefones_convertidos_jid
}


In [20]:
telefones_convertidos_jid

['5527992027535@s.whatsapp.net',
 '5524981604432@s.whatsapp.net',
 '553196856817@s.whatsapp.net',
 '553196161702@s.whatsapp.net',
 '553197733587@s.whatsapp.net',
 '553190861806@s.whatsapp.net',
 '555599666832@s.whatsapp.net',
 '553172144220@s.whatsapp.net',
 '553183993170@s.whatsapp.net',
 '553188683543@s.whatsapp.net',
 '5527997374177@s.whatsapp.net',
 '553171487572@s.whatsapp.net',
 '553171700813@s.whatsapp.net',
 '553173438496@s.whatsapp.net',
 '553173451644@s.whatsapp.net',
 '553183160550@s.whatsapp.net',
 '553185588939@s.whatsapp.net',
 '553188929472@s.whatsapp.net',
 '553189027712@s.whatsapp.net',
 '553190670771@s.whatsapp.net',
 '553191004550@s.whatsapp.net',
 '553192125169@s.whatsapp.net',
 '553194667082@s.whatsapp.net',
 '553195693255@s.whatsapp.net',
 '553197013863@s.whatsapp.net',
 '553197960044@s.whatsapp.net',
 '553198795412@s.whatsapp.net',
 '556181353474@s.whatsapp.net',
 '553199823391@s.whatsapp.net',
 '553199924241@s.whatsapp.net',
 '553288071604@s.whatsapp.net',
 '553

In [132]:
check_phone_has_messages_raw("https://evolutionapi.gbstudios.tech",
                 "Fa Maia Comercial",
                 "VpN4MchD6Y0Z7sWRNHSeViu8emRQR3zm", "5531995655690")   

False

<h1> Envio de Mensagens </h1>

In [23]:
mensagem_1 = """Buenas, tudo bem? 
Aqui é a Fabricia, mentora lá dos grupos Masterclass em Vendas. Fico feliz com sua participação!

Não sei se você sabe, mas trabalho na área de vendas há anos e hoje cuido de algumas empresas com estratégias em crescimento de negócios e profissionais da área da saúde.

Estou tirando um tempinho para conhecer mais de vocês também, quero preparar conteúdos que realmente façam sentido para vocês!

Hoje você trabalha com o que?"""

mensagem_2 = """Buenas, beleza? 
Fabricia aqui, sua mentora do Masterclass em Vendas. Adorei te ver participando!

Não sei se você já me conhece, mas trabalho com vendas há muitos anos e hoje foco em ajudar empresas e profissionais de saúde a estruturarem suas estratégias comerciais.

Quero bater um papo com vocês para conhecer melhor quem está aqui, assim consigo desenvolver conteúdos que façam diferença real na prática de cada um!

O que você faz profissionalmente no momento?"""

mensagem_3 = """Buenas, tudo joia? 
Fabricia falando! Sou mentora do grupo Masterclass em Vendas e estou muito feliz de ver você aqui!

Caso não me conheça ainda, tenho uma longa trajetória em vendas e hoje dedico meu tempo a orientar negócios e profissionais da área da saúde no crescimento de suas vendas.

Estou dando uma passadinha para conhecer vocês melhor, porque quero criar conteúdos que realmente batam com as necessidades de cada um aqui do grupo!

Atualmente você trabalha em que área?"""

mensagem_4 = """Buenas, tudo tranquilo? 
Aqui quem fala é a Fabricia, mentora do Masterclass em Vendas. Super feliz em ver sua participação por aqui!

Talvez você ainda não me conheça, mas atuo com vendas há anos e hoje trabalho diretamente com empresas e profissionais de saúde, ajudando no desenvolvimento de estratégias comerciais.

Resolvi passar aqui para trocar uma ideia com vocês, quero entender o perfil de quem está no grupo para criar materiais que realmente façam sentido para a realidade de vocês!

Me fala aí, você atua em qual segmento hoje?"""

mensagem_5 = """Buenas, como você está? 
Sou a Fabricia, mentora aqui dos grupos Masterclass em Vendas. Que alegria ter você participando!

Pode ser que você não me conheça ainda, mas venho da área de vendas há bastante tempo e hoje me dedico a apoiar empresas e profissionais da saúde com estratégias para alavancar seus resultados.

Estou passando para conversar com vocês e entender melhor quem são, assim posso preparar conteúdos que realmente agreguem e façam diferença no dia a dia de cada um!

Hoje você trabalha com o que exatamente?"""

In [27]:
[send_message("5531995655690", mensagem_5)]

[{'message_result': {'success': True,
   'status_code': 201,
   'message': 'Mensagem enviada com sucesso',
   'data': {'key': {'remoteJid': '553195655690@s.whatsapp.net',
     'fromMe': True,
     'id': '3EB06CA5BC7607E117EC3DF11399B7D7017805EB'},
    'pushName': 'Você',
    'status': 'PENDING',
    'message': {'conversation': 'Buenas, como você está? \nSou a Fabricia, mentora aqui dos grupos Masterclass em Vendas. Que alegria ter você participando!\n\nPode ser que você não me conheça ainda, mas venho da área de vendas há bastante tempo e hoje me dedico a apoiar empresas e profissionais da saúde com estratégias para alavancar seus resultados.\n\nEstou passando para conversar com vocês e entender melhor quem são, assim posso preparar conteúdos que realmente agreguem e façam diferença no dia a dia de cada um!\n\nHoje você trabalha com o que exatamente?'},
    'contextInfo': None,
    'messageType': 'conversation',
    'messageTimestamp': 1756385603,
    'instanceId': 'b8d32ca8-6d3d-4902-